In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 10      # number of variables
num_ineq = 10     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_exact_10-10.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:27<00:00,  4.81it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     -2.752972               0.0      0.206351
std       1.233259               0.0      0.020654
min      -5.254417               0.0      0.153796
25%      -3.663621               0.0      0.198585
50%      -2.827983               0.0      0.201521
75%      -1.934024               0.0      0.216266
max       1.447501               0.0      0.324365
Number of infeasible solution: 0


In [7]:
df = pd.read_csv("result/cq_exact_10-10.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    -2.778616               0.0      0.199638
std      1.399711               0.0      0.013954
min     -5.245462               0.0      0.167444
25%     -3.973540               0.0      0.198235
50%     -2.990693               0.0      0.200515
75%     -1.839966               0.0      0.202987
max      1.447501               0.0      0.263747
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_10-10.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:25<00:00,  6.89it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -2.831070          0.177637      0.137060
std       1.211896          0.087335      0.018610
min      -5.315227          0.000000      0.115648
25%      -3.743787          0.113057      0.131130
50%      -2.897332          0.165136      0.132988
75%      -2.035999          0.227713      0.135511
max       1.287201          0.487457      0.347112
Number of infeasible solution: 999


In [8]:
df = pd.read_csv("result/cq_heur_rnd_10-10.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -2.857766          0.184249      0.139806
std      1.380641          0.095250      0.015732
min     -5.295697          0.016281      0.118579
25%     -4.053361          0.114362      0.132154
50%     -3.033066          0.162895      0.133319
75%     -1.992545          0.255210      0.143576
max      1.287201          0.403525      0.201784
Number of infeasible solution: 100


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_n1_10-10.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:19<00:00,  7.16it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.289597e+15               0.0      0.138138
std    5.794443e+15               0.0      0.043513
min   -5.159240e+00               0.0      0.119536
25%   -3.066814e+00               0.0      0.123953
50%   -2.032727e+00               0.0      0.137413
75%    2.832876e+00               0.0      0.139122
max    4.525135e+16               0.0      0.901425
Number of infeasible solution: 0


In [9]:
df = pd.read_csv("result/cq_heur_n1_10-10.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02             100.0    100.000000
mean   1.219108e+15               0.0      0.132198
std    5.757495e+15               0.0      0.009339
min   -4.765557e+00               0.0      0.120835
25%   -3.147985e+00               0.0      0.123503
50%   -1.900374e+00               0.0      0.136191
75%    5.268976e+00               0.0      0.138600
max    3.688209e+16               0.0      0.168422
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 248.32
Epoch 1, Validation Loss: 83.96
Epoch 2, Validation Loss: 11.49
Epoch 3, Validation Loss: 5.05
Epoch 4, Validation Loss: 4.89
Epoch 5, Validation Loss: 3.85
Epoch 6, Validation Loss: 3.27
Epoch 7, Validation Loss: 4.43
Epoch 8, Validation Loss: 3.47
Epoch 9, Validation Loss: 1.81
Epoch 10, Validation Loss: 1.56
Epoch 11, Validation Loss: 1.02
Epoch 12, Validation Loss: 1.08
Epoch 13, Validation Loss: 0.71
Epoch 14, Validation Loss: 1.40
Epoch 15, Validation Loss: 1.11
Epoch 16, Validation Loss: 0.40
Epoch 17, Validation Loss: 0.40
Epoch 18, Validation Loss: 0.41
Epoch 19, Validation Loss: 1.19
Epoch 20, Validation Loss: 0.74
Epoch 21, Validation Loss: 0.75
Epoch 22, Validation Loss: 0.37
Epoch 23, Validation Loss: -0.17
Epoch 24, Validation Loss: -0.30
Epoch 25, Validation Loss: -0.20
Epoch 26, Validation Loss: -0.52
Epoch 27, Validation Loss: -0.38
Epoch 28, Validation Loss: -0.32
Epoch 29, Validation Loss: -0.56
Epoch 30, Validation Loss: -0.59
Epoch 

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_10-10.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 203.54it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -1.602293          0.000358      0.003116
std      1.672147          0.002678      0.001334
min     -4.187225          0.000000      0.001228
25%     -2.992502          0.000000      0.002016
50%     -1.999704          0.000000      0.002888
75%     -0.622180          0.000000      0.003527
max      3.252418          0.024279      0.008123
Number of infeasible solution: 2


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 253.56
Epoch 1, Validation Loss: 89.36
Epoch 2, Validation Loss: 5.90
Epoch 3, Validation Loss: 3.70
Epoch 4, Validation Loss: 3.06
Epoch 5, Validation Loss: 3.05
Epoch 6, Validation Loss: 2.70
Epoch 7, Validation Loss: 1.97
Epoch 8, Validation Loss: 1.86
Epoch 9, Validation Loss: 2.75
Epoch 10, Validation Loss: 1.65
Epoch 11, Validation Loss: 2.01
Epoch 12, Validation Loss: 2.18
Epoch 13, Validation Loss: 1.67
Epoch 14, Validation Loss: 1.23
Epoch 15, Validation Loss: 1.69
Epoch 16, Validation Loss: 1.98
Epoch 17, Validation Loss: 1.31
Epoch 18, Validation Loss: 0.96
Epoch 19, Validation Loss: 0.88
Epoch 20, Validation Loss: 0.56
Epoch 21, Validation Loss: 0.37
Epoch 22, Validation Loss: 0.70
Epoch 23, Validation Loss: -0.06
Epoch 24, Validation Loss: 0.32
Epoch 25, Validation Loss: 0.33
Epoch 26, Validation Loss: -0.16
Epoch 27, Validation Loss: -0.37
Epoch 28, Validation Loss: -0.51
Epoch 29, Validation Loss: -0.51
Epoch 30, Validation Loss: -0.20
Epoch 31,

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_10-10.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 210.11it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -1.817698          0.001271      0.003038
std      1.652370          0.005377      0.001021
min     -4.591527          0.000000      0.001505
25%     -3.265236          0.000000      0.002008
50%     -2.078343          0.000000      0.003000
75%     -0.703937          0.000000      0.003513
max      3.096604          0.031964      0.006009
Number of infeasible solution: 7


## Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 258.35
Epoch 1, Validation Loss: 96.46
Epoch 2, Validation Loss: 6.20
Epoch 3, Validation Loss: 4.04
Epoch 4, Validation Loss: 3.37
Epoch 5, Validation Loss: 2.73
Epoch 6, Validation Loss: 2.24
Epoch 7, Validation Loss: 2.03
Epoch 8, Validation Loss: 2.04
Epoch 9, Validation Loss: 1.84
Epoch 10, Validation Loss: 1.65
Epoch 11, Validation Loss: 1.38
Epoch 12, Validation Loss: 0.74
Epoch 13, Validation Loss: 0.58
Epoch 14, Validation Loss: 0.53
Epoch 15, Validation Loss: 0.13
Epoch 16, Validation Loss: 0.13
Epoch 17, Validation Loss: -0.00
Epoch 18, Validation Loss: -0.03
Epoch 19, Validation Loss: -0.08
Epoch 20, Validation Loss: -0.14
Epoch 21, Validation Loss: 0.40
Epoch 22, Validation Loss: -0.42
Epoch 23, Validation Loss: -0.68
Epoch 24, Validation Loss: -0.52
Epoch 25, Validation Loss: -0.42
Epoch 26, Validation Loss: -0.34
Epoch 27, Validation Loss: -0.88
Epoch 28, Validation Loss: -1.06
Epoch 29, Validation Loss: -1.13
Epoch 30, Validation Loss: -1.14
Ep

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_pr_10-10.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 370.58it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    -2.481321          0.078485      0.000698
std      1.548409          0.057827      0.000486
min     -4.868193          0.000000      0.000000
25%     -3.808517          0.032311      0.000000
50%     -2.755608          0.069870      0.001000
75%     -1.583942          0.111662      0.001006
max      2.281136          0.262004      0.001507
Number of infeasible solution: 93


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 253.56
Epoch 1, Validation Loss: 108.38
Epoch 2, Validation Loss: 20.31
Epoch 3, Validation Loss: 5.15
Epoch 4, Validation Loss: 4.24
Epoch 5, Validation Loss: 4.20
Epoch 6, Validation Loss: 3.94
Epoch 7, Validation Loss: 3.94
Epoch 8, Validation Loss: 3.90
Epoch 9, Validation Loss: 3.96
Epoch 10, Validation Loss: 3.84
Epoch 11, Validation Loss: 3.59
Epoch 12, Validation Loss: 3.51
Epoch 13, Validation Loss: 3.40
Epoch 14, Validation Loss: 3.48
Epoch 15, Validation Loss: 2.96
Epoch 16, Validation Loss: 2.70
Epoch 17, Validation Loss: 2.04
Epoch 18, Validation Loss: 2.02
Epoch 19, Validation Loss: 1.69
Epoch 20, Validation Loss: 1.71
Epoch 21, Validation Loss: 1.63
Epoch 22, Validation Loss: 1.82
Epoch 23, Validation Loss: 1.46
Epoch 24, Validation Loss: 1.62
Epoch 25, Validation Loss: 1.50
Epoch 26, Validation Loss: 1.57
Epoch 27, Validation Loss: 1.49
Epoch 28, Validation Loss: 1.42
Epoch 29, Validation Loss: 1.59
Epoch 30, Validation Loss: 1.65
Epoch 31, Val

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_st_10-10.csv")

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 262.28it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    -0.942166               0.0      0.001851
std      1.880038               0.0      0.000653
min     -3.665615               0.0      0.000000
25%     -2.462619               0.0      0.001128
50%     -1.319264               0.0      0.002008
75%      0.266201               0.0      0.002292
max      4.683797               0.0      0.003017
Number of infeasible solution: 0
